# Laboratorio 4 - Familias de Malware

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Parte 1
### Creación del dataset
Para la creación del dataset se analizaron las muestras de malware en una máquina virtual con kali linux. En esta máquina virtual se descomprimió el archivo MALWR.zip y se desempaquetaron los ejecutables con upx. Luego se creó un programa en python para convertir las llamadas al API de los malwares a un csv. Este programa se llama `peheaders.py` y es una adaptación del programa proporcionado en la plataforma. Para introducir los datos de cada malware se ejecutó el siguiente comando en bash (los malwares estan en una carpeta llamada MALWR):

```bash
for filename in MALWR/*
do
python3 peheaders.py $filename
done
```
En el lugar que se ejecuta este comando se guarda un archivo llamado `dataset.csv` el cual es usado en el resto del laboratorio.

### Exploración y pre-procesamiento de datos

In [6]:
df = pd.read_csv("dataset.csv")
df.head(10)

,file,api
0,1F2EB7B090018D975E6D9B40868C94CA,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
1,6FAA4740F99408D4D2DDDD0B09BBDEFD,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
2,33DE5067A433A6EC5C328067DC18EC37,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
3,99A39866A657A10949FCB6D634BB30D5,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
4,650A6FCA433EE243391E4B4C11F09438,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
5,8442AE37B91F279A9F06DE4C60B286A3,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
6,65018CD542145A3792BA09985734C12A,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
7,785003A405BC7A4EBCBB21DDB757BF3F,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
8,A316D5AECA269CA865077E7FFF356E7D,"CloseHandle,WaitForSingleObject,CreateEventA,E..."
9,AAAz2E1B6940985A23E5639450F8391820655,"GetVersionExA,FreeLibrary,GetWindowsDirectoryA..."


In [7]:
df = df.drop("file", axis=1)
body = np.array(df['api'])

In [9]:
cv = CountVectorizer(min_df=0.001)
cv_matrix = cv.fit_transform(body)
cv_matrix = cv_matrix.toarray()
vocabulario = cv.get_feature_names_out()
vocabulario.shape
bow = pd.DataFrame(cv_matrix, columns=vocabulario)
bow.head(10)

,1type_info,__cxxframehandler,__dllonexit,__getmainargs,__p___initenv,__p__commode,__p__fmode,__set_app_type,__setusermatherr,_acmdln,...,wcstombs,writefile,wsacleanup,wsagetlasterror,wsasetlasterror,wsastartup,wsprintfa,xz,yapaxi,yaxpax
0,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
1,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
2,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
3,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
4,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
5,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
6,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
7,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
8,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
9,1,1,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,1


## Parte 2
### Implementación del modelo

### Conclusiones

1. ¿Para qué número de clústeres se obtiene el coeficiente de Silhouette más alto?
2. ¿Coincide el coeficiente de Silhouette con el método del codo?
3. ¿Cuántas familias existen entre los ejemplares de malware proporcionados?
4. ¿Coincide el índice de Jaccard con las familias encontradas?